In [ ]:
import sys
import os
import glob
sys.path.append('../') # Add parent directory to path to find proto modules

from google.protobuf.internal.decoder import _DecodeVarint32

# Ensure the proto module is importable
try:
    from proto import segmentation_pb2
except ImportError:
    # Fallback if proto is not a package (no __init__.py)
    sys.path.append('../proto')
    import segmentation_pb2

def read_delimited_messages(filename, message_class):
    messages = []
    with open(filename, 'rb') as f:
        buf = f.read()
        n = 0
        while n < len(buf):
            msg_len, new_pos = _DecodeVarint32(buf, n)
            n = new_pos
            msg_buf = buf[n:n+msg_len]
            n += msg_len
            message = message_class()
            message.ParseFromString(msg_buf)
            messages.append(message)
    return messages

# Find likely .seg.pb files in ../recordings/
search_path = '../recordings/*.seg.pb'
files = glob.glob(search_path)

if not files:
    print(f"No .seg.pb files found in {search_path}")
    seg_pb_path = 'segmentation_output.seg.pb' # Fallback default
else:
    # Use the most recent file
    seg_pb_path = max(files, key=os.path.getmtime)
    print(f"Using most recent file: {seg_pb_path}")

if os.path.exists(seg_pb_path):
    try:
        responses = read_delimited_messages(seg_pb_path, segmentation_pb2.SegmentationResponse)
        print(f"Successfully read {len(responses)} SegmentationResponse messages from {seg_pb_path}.")
        
        if responses:
            print(f"\nFirst response details:\n{responses[0]}")
    except Exception as e:
        print(f"Error reading file: {e}")
else:
    print(f"File '{seg_pb_path}' not found.")